In [1]:
full_data <- read.csv("./../data/donnees-synop-essentielles-omm.csv", 
            sep=";", 
            na.string=c("","null","NaN"),
            header=TRUE, 
            stringsAsFactors=TRUE) # SI IL Y A DES PROBLÈMES BIZARRES PLUS LOIN, METTRE À FALSE

# On se demande si les moyennes de température mensuelles diffèrent significativement entre les régions.

In [8]:
library(dplyr)

temp_mensuelle <- full_data %>%
  mutate(Date_only = as.Date(substr(Date, 1, 10)),
         month_only = as.Date(paste0(format(Date_only, "%Y-%m"), "-01"))) %>%
  group_by(month_only, region..name.) %>%
  summarise(mean_temp = mean(Température...C., na.rm = TRUE), .groups = "drop")

In [9]:
anova_model <- aov(mean_temp ~ region..name., data = temp_mensuelle)
summary(anova_model)

# Diagnostics
shapiro.test(residuals(anova_model))   # normalité
car::leveneTest(mean_temp ~ region..name., data = temp_mensuelle)  # variances

                Df Sum Sq Mean Sq F value Pr(>F)    
region..name.   20  98863    4943   208.8 <2e-16 ***
Residuals     2433  57603      24                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
125 observations effacées parce que manquantes


	Shapiro-Wilk normality test

data:  residuals(anova_model)
W = 0.98683, p-value = 2.73e-14


,Df,F value,Pr(>F)
,<int>,<dbl>,<dbl>
group,20,78.99536,1.650646e-246
,2433,NA,NA


Le résultats du test n'est pas très significatif. Les résidus ne sont pas Gaussiens. Ça doit être dû au faite que la température varie principalment en fonction de la saison.

On va donc se ramener à une granularité inférieur, en regardant si pour chaque mois la température journalière moyenne dans chaque saison varie en fonction de la region. 

In [22]:
library(dplyr)
library(broom)
library(ggplot2)

# On s'assure que month_label est présent dans temp_journaliere
temp_journaliere <- temp_journaliere %>%
  mutate(month_label = format(Date_only, "%Y-%m"))

# Split par mois
temp_by_month <- temp_journaliere %>%
  group_split(month_label)

# Fonction qui teste ANOVA puis Kruskal
test_one_month <- function(df) {
  aov_model <- aov(mean_temp ~ region..name., data = df)
  
  shapiro_p <- tryCatch(
    shapiro.test(residuals(aov_model))$p.value,
    error = function(e) NA
  )
  
  if (!is.na(shapiro_p) && shapiro_p > 0.05) {
    # ANOVA valide
    res <- tidy(aov_model) %>% mutate(test = "ANOVA", shapiro_p = shapiro_p)
  } else {
    # Sinon Kruskal
    kw <- kruskal.test(mean_temp ~ region..name., data = df)
    res <- tibble(
      term = "region..name.",
      statistic = kw$statistic,
      p.value = kw$p.value,
      test = "Kruskal-Wallis",
      shapiro_p = shapiro_p
    )
  }
  
  # On ajoute le mois ici
  res <- res %>% mutate(month_label = unique(df$month_label))
  
  return(res)
}

# Appliquer sur chaque mois
results_by_month <- map_dfr(temp_by_month, test_one_month)


results_by_month

term,statistic,p.value,test,shapiro_p,month_label
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
region..name.,382.8227,1.624978e-69,Kruskal-Wallis,6.220663e-03,2015-08
region..name.,420.4892,9.568923e-79,Kruskal-Wallis,2.811436e-07,2015-09
region..name.,412.4457,4.622670e-77,Kruskal-Wallis,2.901195e-09,2015-10
region..name.,359.0704,6.400226e-66,Kruskal-Wallis,2.235842e-11,2015-11
region..name.,410.9365,9.565802e-77,Kruskal-Wallis,6.589644e-09,2015-12
region..name.,394.7132,2.360747e-73,Kruskal-Wallis,1.971818e-12,2016-01
region..name.,371.6567,1.530065e-68,Kruskal-Wallis,2.206547e-02,2016-02
region..name.,412.9973,3.543635e-77,Kruskal-Wallis,6.929432e-08,2016-03
region..name.,408.0547,3.834415e-76,Kruskal-Wallis,3.349008e-02,2016-04


Au final c'est le même schéma en mesurant les moyenne journalière sur un mois. L'hypothèse des résidus gaussien ne tient pas.  